# **OBTENCIÓN DE DATOS METEOROLÓGICOS**

Este notebook obtiene datos meteorológicos históricos para la ubicación de la Presa Abelardo L. Rodríguez desde la API de Open-Meteo.

**Coordenadas de la presa:**
- Latitud: 29.067999°N (29.1026°N en versión simplificada)
- Longitud: -110.910734°W (-110.9773°W en versión simplificada)

**Salidas:** 
- `../data/raw/datos_meteorologicos_basicos.csv` (evapotranspiración)
- `../data/raw/datos_meteorologicos_completos.csv` (variables múltiples)

## 1. Importar librerías

In [1]:
import pandas as pd
import requests
import time
import os
from datetime import datetime
from pathlib import Path

# Definir rutas del proyecto siguiendo estructura Cookiecutter
project_dir = Path.cwd().parent
data_dir = project_dir / 'data' / 'raw'

print(f"Directorio de datos: {data_dir}")

Directorio de datos: C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\raw


## 2. Configuración de coordenadas y fechas

In [2]:
# Coordenadas de la Presa Abelardo L. Rodríguez
LATITUD = 29.067999
LONGITUD = -110.910734

# Rango de fechas (ajustar según necesidad)
FECHA_INICIO = '1940-01-01'
FECHA_FIN = '2024-12-31'

print(f"Coordenadas: ({LATITUD}, {LONGITUD})")
print(f"Rango de fechas: {FECHA_INICIO} a {FECHA_FIN}")

Coordenadas: (29.067999, -110.910734)
Rango de fechas: 1940-01-01 a 2024-12-31


## 3. Opción 1: Datos básicos (Evapotranspiración)

Obtenemos únicamente la evapotranspiración de referencia (ET0 FAO) que es fundamental para el análisis hidrológico.

In [3]:
# URL para datos básicos: solo evapotranspiración
API_URL_BASICO = (
    f'https://archive-api.open-meteo.com/v1/archive?'
    f'latitude={LATITUD}&longitude={LONGITUD}&'
    f'start_date={FECHA_INICIO}&end_date={FECHA_FIN}&'
    f'daily=et0_fao_evapotranspiration&'
    f'timezone=America/Los_Angeles'
)

print("Obteniendo datos de evapotranspiración...")
response = requests.get(API_URL_BASICO)

if response.status_code == 200:
    data_basico = response.json()
    print("✓ Datos obtenidos exitosamente")
    
    # Crear DataFrame
    df_evapo = pd.DataFrame(data_basico['daily'])
    df_evapo["time"] = pd.to_datetime(df_evapo["time"])
    df_evapo.columns = ['fecha', 'evapotranspiracion_mm_dia']
    
    print(f"Registros obtenidos: {len(df_evapo)}")
    print(f"Rango: {df_evapo['fecha'].min()} a {df_evapo['fecha'].max()}")
    display(df_evapo.head())
    
else:
    print(f"✗ Error al obtener datos: {response.status_code}")
    if response.status_code == 429:
        print("  Razón: Límite de peticiones excedido. Espere unos minutos e intente de nuevo.")
    df_evapo = None

Obteniendo datos de evapotranspiración...
✓ Datos obtenidos exitosamente
Registros obtenidos: 31047
Rango: 1940-01-01 00:00:00 a 2024-12-31 00:00:00


,fecha,evapotranspiracion_mm_dia
0,1940-01-01,3.19
1,1940-01-02,2.86
2,1940-01-03,3.68
3,1940-01-04,2.75
4,1940-01-05,2.19


### 3.1 Guardar datos básicos en CSV

In [4]:
if df_evapo is not None:
    output_file = data_dir / 'datos_meteorologicos_basicos.csv'
    df_evapo.to_csv(output_file, index=False)
    
    print(f"✓ Archivo guardado: {output_file}")
    print(f"  - Registros: {len(df_evapo)}")
    print(f"  - Columnas: {list(df_evapo.columns)}")
    print(f"  - Tamaño: {os.path.getsize(output_file) / 1024:.2f} KB")
    
    # Estadísticas
    print("\nEstadísticas de evapotranspiración (mm/día):")
    print(df_evapo['evapotranspiracion_mm_dia'].describe())
else:
    print("No se pudo guardar el archivo porque no se obtuvieron datos.")

✓ Archivo guardado: C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\raw\datos_meteorologicos_basicos.csv
  - Registros: 31047
  - Columnas: ['fecha', 'evapotranspiracion_mm_dia']
  - Tamaño: 512.62 KB

Estadísticas de evapotranspiración (mm/día):
count    31047.000000
mean         5.727098
std          2.059888
min          0.370000
25%          4.060000
50%          5.670000
75%          7.375000
max         13.290000
Name: evapotranspiracion_mm_dia, dtype: float64


## 4. Datos meteorológicos completos

Obtenemos un conjunto completo de variables meteorológicas para análisis hidrológico detallado:

**Variables incluidas:**
- Precipitación (total y lluvia) y horas de precipitación
- Evapotranspiración
- Temperatura (media, máxima, mínima)
- Viento (máximo y medio)
- Déficit de presión de vapor
- Radiación solar
- Humedad relativa
- Cobertura de nubes
- Humedad del suelo (4 profundidades: 0-7cm, 7-28cm, 28-100cm, 0-100cm)

In [5]:
# URL para datos meteorológicos completos
API_URL_COMPLETO = (
    "https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={LATITUD}&longitude={LONGITUD}&"
    f"start_date={FECHA_INICIO}&end_date={FECHA_FIN}&"
    "daily=precipitation_sum,rain_sum,et0_fao_evapotranspiration,"
    "temperature_2m_mean,temperature_2m_max,temperature_2m_min,"
    "precipitation_hours,wind_speed_10m_max,wind_speed_10m_mean,"
    "vapour_pressure_deficit_max,shortwave_radiation_sum,"
    "relative_humidity_2m_mean,cloud_cover_mean,"
    "soil_moisture_0_to_100cm_mean,soil_moisture_0_to_7cm_mean,"
    "soil_moisture_28_to_100cm_mean,soil_moisture_7_to_28cm_mean&"
    "timezone=auto"
)

print("Obteniendo datos meteorológicos completos...")
print(f"Rango: {FECHA_INICIO} a {FECHA_FIN}")
print("Variables: 17")
print("\n(Esto puede tardar 10-30 segundos)\n")

try:
    response_completo = requests.get(API_URL_COMPLETO, timeout=60)
    
    if response_completo.status_code == 200:
        # Verificar si la respuesta contiene un mensaje de error
        if len(response_completo.content) < 100:
            error_msg = response_completo.text
            if "timeout" in error_msg.lower() or "error" in error_msg.lower():
                print(f"✗ Error del servidor: {error_msg}")
                print("\n💡 Solución: La solicitud es muy grande.")
                print("   Opción 1: Reduce el rango de fechas en la celda 4")
                print("   Opción 2: Usa solo datos básicos (ya guardados)")
                df_meteo_completo = None
            else:
                raise ValueError("Respuesta inesperada del servidor")
        else:
            # Parsear JSON
            data_completo = response_completo.json()
            
            if 'daily' in data_completo:
                # Crear DataFrame
                df_meteo_completo = pd.DataFrame(data_completo['daily'])
                df_meteo_completo['time'] = pd.to_datetime(df_meteo_completo['time'])
                
                # Renombrar columnas
                df_meteo_completo.columns = [
                    'fecha',
                    'precipitacion_mm',
                    'lluvia_mm',
                    'evapotranspiracion_mm',
                    'temp_media_c',
                    'temp_max_c',
                    'temp_min_c',
                    'horas_precipitacion',
                    'viento_max_km_h',
                    'viento_medio_km_h',
                    'deficit_presion_vapor_kpa',
                    'radiacion_solar_mj_m2',
                    'humedad_relativa_pct',
                    'cobertura_nubes_pct',
                    'humedad_suelo_0_100cm',
                    'humedad_suelo_0_7cm',
                    'humedad_suelo_28_100cm',
                    'humedad_suelo_7_28cm'
                ]
                
                print(f"✓ Datos obtenidos exitosamente")
                print(f"  Registros: {len(df_meteo_completo):,}")
                print(f"  Variables: {len(df_meteo_completo.columns)}")
                print(f"  Rango: {df_meteo_completo['fecha'].min().date()} a {df_meteo_completo['fecha'].max().date()}")
                print(f"\nPrimeros registros:")
                display(df_meteo_completo.head())
                
            else:
                print("✗ Error: La respuesta no contiene datos 'daily'")
                print(f"  Claves encontradas: {list(data_completo.keys())}")
                df_meteo_completo = None
                
    elif response_completo.status_code == 429:
        print("✗ Error 429: Límite de peticiones alcanzado")
        print("\n💡 Soluciones:")
        print("  - Límites API gratuita: 600/min, 5,000/hora, 10,000/día")
        print("  - Espera 1 hora e intenta nuevamente")
        print("  - Mientras tanto, usa datos básicos (evapotranspiración)")
        df_meteo_completo = None
        
    else:
        print(f"✗ Error HTTP {response_completo.status_code}")
        print(f"  Respuesta: {response_completo.text[:200]}")
        df_meteo_completo = None
        
except requests.exceptions.Timeout:
    print("✗ Timeout: La solicitud tardó más de 60 segundos")
    print("\n💡 Soluciones:")
    print("  1. Intenta nuevamente (puede ser temporal)")
    print("  2. Reduce el rango de fechas en la celda 4")
    print("  3. Usa datos básicos (evapotranspiración) ya guardados")
    df_meteo_completo = None
    
except Exception as e:
    print(f"✗ Error inesperado: {type(e).__name__}: {e}")
    df_meteo_completo = None

Obteniendo datos meteorológicos completos...
Rango: 1940-01-01 a 2024-12-31
Variables: 17

(Esto puede tardar 10-30 segundos)

✓ Datos obtenidos exitosamente
  Registros: 31,047
  Variables: 18
  Rango: 1940-01-01 a 2024-12-31

Primeros registros:


,fecha,precipitacion_mm,lluvia_mm,evapotranspiracion_mm,temp_media_c,temp_max_c,temp_min_c,horas_precipitacion,viento_max_km_h,viento_medio_km_h,deficit_presion_vapor_kpa,radiacion_solar_mj_m2,humedad_relativa_pct,cobertura_nubes_pct,humedad_suelo_0_100cm,humedad_suelo_0_7cm,humedad_suelo_28_100cm,humedad_suelo_7_28cm
0,1940-01-01,0.0,0.0,3.19,15.5,23.7,7.9,0.0,11.9,6.8,2.33,14.54,42,41,0.053,0.053,0.051,0.062
1,1940-01-02,0.0,0.0,2.86,15.5,23.3,8.4,0.0,13.0,7.2,2.30,11.71,42,77,0.053,0.053,0.051,0.061
2,1940-01-03,0.0,0.0,3.68,17.0,25.5,9.5,0.0,18.2,9.7,2.60,14.10,42,63,0.053,0.052,0.051,0.061
3,1940-01-04,0.0,0.0,2.75,16.1,22.8,9.2,0.0,13.2,6.5,2.00,11.94,52,68,0.053,0.052,0.051,0.061
4,1940-01-05,0.1,0.1,2.19,15.1,21.0,10.0,1.0,19.8,11.3,1.38,8.83,66,78,0.053,0.052,0.051,0.061


### 4.1 Guardar datos completos en CSV

In [6]:
if df_meteo_completo is not None:
    output_file = data_dir / 'datos_meteorologicos_completos.csv'
    df_meteo_completo.to_csv(output_file, index=False)
    
    print(f"✓ Archivo guardado: {output_file}")
    print(f"  - Registros: {len(df_meteo_completo)}")
    print(f"  - Columnas: {len(df_meteo_completo.columns)}")
    print(f"  - Tamaño: {os.path.getsize(output_file) / 1024:.2f} KB")
    
    # Estadísticas descriptivas
    print("\nEstadísticas descriptivas (primeras 5 variables):")
    display(df_meteo_completo.iloc[:, 1:6].describe())
    
    # Información de valores faltantes
    print("\nValores faltantes por columna:")
    missing = df_meteo_completo.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("No hay valores faltantes")
else:
    print("No se pudo guardar el archivo porque no se obtuvieron datos.")

✓ Archivo guardado: C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\raw\datos_meteorologicos_completos.csv
  - Registros: 31047
  - Columnas: 18
  - Tamaño: 2821.31 KB

Estadísticas descriptivas (primeras 5 variables):


,precipitacion_mm,lluvia_mm,evapotranspiracion_mm,temp_media_c,temp_max_c
count,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000
mean,0.564502,0.564502,5.727098,24.190466,31.442281
std,2.721520,2.721520,2.059888,6.658277,6.729381
min,0.000000,0.000000,0.370000,3.500000,8.400000
25%,0.000000,0.000000,4.060000,18.500000,26.400000
50%,0.000000,0.000000,5.670000,24.600000,32.100000
75%,0.000000,0.000000,7.375000,30.000000,36.900000
max,87.900000,87.900000,13.290000,39.200000,47.100000



Valores faltantes por columna:
No hay valores faltantes


## 5. Resumen de archivos generados

In [7]:
print("="*60)
print("RESUMEN DE ARCHIVOS GENERADOS")
print("="*60)

archivos_generados = []

# Verificar archivos generados
file_basico = data_dir / 'datos_meteorologicos_basicos.csv'
file_completo = data_dir / 'datos_meteorologicos_completos.csv'

if file_basico.exists():
    archivos_generados.append(file_basico.name)
    print(f"\n✓ {file_basico}")
    print("  Contiene: Evapotranspiración diaria")
    print(f"  Tamaño: {os.path.getsize(file_basico) / 1024:.2f} KB")

if file_completo.exists():
    archivos_generados.append(file_completo.name)
    print(f"\n✓ {file_completo}")
    print("  Contiene: 17 variables meteorológicas")
    print(f"  Tamaño: {os.path.getsize(file_completo) / 1024:.2f} KB")

if len(archivos_generados) == 0:
    print("\n✗ No se generaron archivos")
    print("  Verifique los errores en las celdas anteriores")
else:
    print(f"\n{len(archivos_generados)} archivo(s) generado(s) exitosamente en data/raw/")

print("\n" + "="*60)
print("PROCESO COMPLETADO")
print("="*60)

RESUMEN DE ARCHIVOS GENERADOS

✓ C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\raw\datos_meteorologicos_basicos.csv
  Contiene: Evapotranspiración diaria
  Tamaño: 512.62 KB

✓ C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\raw\datos_meteorologicos_completos.csv
  Contiene: 17 variables meteorológicas
  Tamaño: 2821.31 KB

2 archivo(s) generado(s) exitosamente en data/raw/

PROCESO COMPLETADO
